In [ ]:
# ref https://analytics4everything.tistory.com/105


In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
#for dirname, _, filenames in os.walk('/kaggle/input'):
#    for filename in filenames:
        #print(os.path.join(dirname, filename))
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import pandas as pd
df = pd.read_csv('../input/h-and-m-personalized-fashion-recommendations/transactions_train.csv')

In [3]:
# 구매 순서 정렬 코드가 가지는 의미가 연결되어 알고리즘이 완성된다
df = df.sort_values(by='t_dat')

In [4]:
# 색인 속도가 너무 느리다
# groupby 하지 않고 customer와 t_dat를 따로 떼어 사용하기
# 빅데이터를 다루는 것 자체만으로 쉬운 건 아니다


In [5]:
%%time
df = df.sort_values(by=['customer_id','t_dat'])

CPU times: user 21.9 s, sys: 1.62 s, total: 23.5 s
Wall time: 23.5 s


In [6]:
customer_t_dat = df[['t_dat','customer_id']].values.tolist()

In [8]:
%%time
for customer_id , t_dat in customer_t_dat:
    print(customer_id,t_dat)
    print(df[(df.customer_id == customer_id) & (df.t_dat ==t_dat)])
    break

2018-12-27 00000dbacae5abe5e23885899a1fa44253a17956c6d1c3d25f88aa139fdfc657
Empty DataFrame
Columns: [t_dat, customer_id, article_id, price, sales_channel_id]
Index: []
CPU times: user 4.62 s, sys: 70.6 ms, total: 4.69 s
Wall time: 4.64 s


In [11]:
customer_ids = df['customer_id'].unique().tolist()

In [10]:
grp_df = df.groupby(['customer_id','t_dat'])

In [17]:
import datetime

In [18]:
day_start = datetime.datetime(2020,9,23)
len_tr = 7
day_string= [day_start - datetime.timedelta(days=i-len_tr+1) for i in range(len_tr)]

datetime.datetime(2020, 9, 23, 0, 0)

In [25]:
 # set of train date
tr_set =  [1,3,5,7,22]

In [26]:
day_start_val = datetime.datetime(2020,9,23)
day_start = [day_start_val - datetime.timedelta(days=i-1+len_tr) for i in tr_set]

In [30]:
day_end = [day_start_val - datetime.timedelta(days=i) for i in tr_set]

In [33]:
# day_start list에서 기간만큼 sampling
q_date = f"((t_dat >= '{day_start[0]}') and (t_dat <= '{day_end[0]}'))"
positive_sample_cust = df.query(q_date)
pos_smp_cust = positive_sample_cust.sort_values(['t_dat','customer_id'])

In [36]:
cand1_dict = {}

In [41]:
day_start[0]

datetime.datetime(2020, 9, 16, 0, 0)

'2020-09-16'

In [54]:
pos_smp_cust[pos_smp_cust.t_dat < datetime.datetime.strftime(day_start[0],"%Y-%m-%d")]

,t_dat,customer_id,article_id,price,sales_channel_id


In [62]:
customer_ids = pos_smp_cust['customer_id'].unique().tolist()
t_dats = pos_smp_cust['t_dat'].unique().tolist()

In [69]:
len(customer_ids)
grp_df = pos_smp_cust.groupby(['customer_id','t_dat'])

In [81]:
pos_smp_cust.groupby(['customer_id','t_dat']).count().reset_index().sort_values(by=['customer_id','t_dat'])[['customer_id','t_dat']].values

array([['00039306476aaf41a07fed942884f16b30abfa83a2a8bea972019098d6406793',
        '2020-09-20'],
       ['0003e867a930d0d6842f923d6ba7c9b77aba33fe2a0fbf4672f30b3e622fec55',
        '2020-09-22'],
       ['000493dd9fc463df1acc2081450c9e75ef8e87d5dd17ed6396773839f6bf71a9',
        '2020-09-21'],
       ...,
       ['fffa67737587e52ff1afa9c7c6490b5eb7acbc439fe82bd11d746ddb223dff26',
        '2020-09-21'],
       ['fffa7d7799eb390a76308454cbdd76e473d65b1497fbe44fe8cf95effea0bed7',
        '2020-09-19'],
       ['fffef3b6b73545df065b521e19f64bf6fe93bfd450ab20e02ce5d1e58a8f700b',
        '2020-09-22']], dtype=object)

In [87]:
grp_df = pos_smp_cust.groupby(['t_dat','customer_id'])

In [95]:
%%time
for group in grp_df:
    article_ids = pd.DataFrame(group[1])['article_id'].values
    for article_id in article_ids:
        # 소비자 그룹별 중복을 제거했으므로 키 검사를 통해 article_id support 값 구하기
        if article_id in cand1_dict.keys():
            cand1_dict[article_id] += 1
        else:
            cand1_dict[article_id] = 1

In [108]:
pruning = int(np.round(sum(cand1_dict.values()) / len(cand1_dict.values()),0))

In [109]:

pruning_cand1 = { key:value       for key,value in cand1_dict.items() if value > pruning}

False

In [137]:
 ''.join(list(article_ids))

TypeError: sequence item 0: expected str instance, numpy.int64 found

In [144]:
item_str = ' '.join(list(map(str,article_ids.tolist())))

In [146]:
item_str

'903762001 879189005'

In [148]:
item_str.index('903762001')

0

In [149]:
item_str.index('879189005')

10

In [133]:
for i, group in grp_df:
    customer_id = i[1]
    if customer_id == pre_customer_id:
        article_ids = pd.DataFrame(group)['article_id'].values
        join_item_str = 
    elif customer_id != pre_customer_id:
        join_item_list = []
        
    break

SyntaxError: invalid syntax (2647467616.py, line 5)

In [121]:
from itertools import permutations
# 순서가 없는 조합
list(permutations(list(pruning_cand1.keys()),2))

[(456163060, 914441004),
 (456163060, 897221001),
 (456163060, 573085057),
 (456163060, 779781013),
 (456163060, 507909001),
 (456163060, 918292001),
 (456163060, 914537001),
 (456163060, 832458002),
 (456163060, 915529001),
 (456163060, 918525001),
 (456163060, 923758001),
 (456163060, 803757004),
 (456163060, 456163090),
 (456163060, 871997002),
 (456163060, 871997001),
 (456163060, 936862001),
 (456163060, 873771001),
 (456163060, 926285002),
 (456163060, 903049003),
 (456163060, 752814022),
 (456163060, 886966002),
 (456163060, 896064001),
 (456163060, 903004003),
 (456163060, 611415035),
 (456163060, 918292003),
 (456163060, 678942058),
 (456163060, 678942001),
 (456163060, 906305002),
 (456163060, 473954013),
 (456163060, 858856005),
 (456163060, 876342001),
 (456163060, 924243001),
 (456163060, 898694001),
 (456163060, 865926002),
 (456163060, 924243002),
 (456163060, 898818003),
 (456163060, 894780001),
 (456163060, 865926001),
 (456163060, 889550001),
 (456163060, 715828015),


In [119]:
from itertools import combinations
# 순서가 없는 조합
list(combinations(list(pruning_cand1.keys()),2))

[(456163060, 914441004),
 (456163060, 897221001),
 (456163060, 573085057),
 (456163060, 779781013),
 (456163060, 507909001),
 (456163060, 918292001),
 (456163060, 914537001),
 (456163060, 832458002),
 (456163060, 915529001),
 (456163060, 918525001),
 (456163060, 923758001),
 (456163060, 803757004),
 (456163060, 456163090),
 (456163060, 871997002),
 (456163060, 871997001),
 (456163060, 936862001),
 (456163060, 873771001),
 (456163060, 926285002),
 (456163060, 903049003),
 (456163060, 752814022),
 (456163060, 886966002),
 (456163060, 896064001),
 (456163060, 903004003),
 (456163060, 611415035),
 (456163060, 918292003),
 (456163060, 678942058),
 (456163060, 678942001),
 (456163060, 906305002),
 (456163060, 473954013),
 (456163060, 858856005),
 (456163060, 876342001),
 (456163060, 924243001),
 (456163060, 898694001),
 (456163060, 865926002),
 (456163060, 924243002),
 (456163060, 898818003),
 (456163060, 894780001),
 (456163060, 865926001),
 (456163060, 889550001),
 (456163060, 715828015),


In [35]:
%%time
from tqdm import tqdm_notebook
for customer_id in tqdm_notebook(customer_ids):
#    print(grp_df.get_group(customer_id).article_id.unique())
    same_customer = True
    article_ids = pos_smp_cust[pos_smp_cust.customer_id == customer_id].article_id.unique()
    for article_id in article_ids:
        # 소비자 그룹별 중복을 제거했으므로 키 검사를 통해 article_id support 값 구하기
        if article_id in cand1_dict.keys():
            cand1_dict[article_id] += 1
        else:
            cand1_dict[article_id] = 1

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


  0%|          | 0/1362281 [00:00<?, ?it/s]

NameError: name 'cand1_dict' is not defined

In [9]:
#  순서가 있는 정방행렬쌍 permutations
for ordered in permutations:
    for item in items:
        if ordered[0] in item and ordered[1] in item:
            
            # 주의 사항 : 순서가 있는 쌍을 순서가 없는 쌍에서 count 하지 않는다 소비자 , 날짜 별로  "()" 괄호로 분리
            if item[item.index(ordered[0])-1] =='(' and item[item.index(ordered[1])+1]  == ')'
    # < > 
#  순서가 없는 정방행렬쌍 combinations
for no_ordered

SyntaxError: invalid syntax (1491622845.py, line 7)

In [ ]:
customer_ids[0]

In [ ]:
cnt = 0

In [ ]:
cand1_dict = {}

In [ ]:
X 

In [ ]:
day_string

In [ ]:
day_end_val = day_start_val + datetime.timedelta(days=6)
q_date = f"((t_dat >= '{day_start[0]}'))"

In [ ]:
day_start

In [ ]:
a

In [ ]:
q_date = f"((t_dat >= '{day_start[0]}'))"

In [ ]:
df.query(q_date)

In [ ]:
day_start

In [ ]:
day_start = [day_start_val - datetime.timedelta(days=i-1+len_tr) for i in tr_set]

In [ ]:
day_end = [day_start_val - datetime.timedelta(days=i) for i in tr_set]

In [ ]:
day_end

In [ ]:
day_start

In [ ]:
cand1_dict = {}

In [ ]:

list_of_buy = [df.query(q_date)]

In [ ]:
# 하위 퍼센티지 백분위 수 계산
#import numpy as np

#arry = np.array([4,6,8,10,12])#

#percentile = np.percentile(arry, 50)

#print("The percentile is:",percentile)

In [ ]:
# 하위 퍼센티지 백분위 수 계산





In [ ]:
customer_ids = pos_smp_cust.customer_id.unique().tolist()

In [ ]:
cand1_dict = {}

In [ ]:
len(customer_ids)
grp_df = pos_smp_cust.groupby(['customer_id','t_dat'])

In [ ]:
# for group in grp_df:
    

In [ ]:
cand1_dict


In [ ]:
# 순서가 있는 경우

min_support = np.percentile(arry, 30)
if key in cand1_dict.keys():
    if cand1_dict[key] < min_support:
        del cand1_dict[key]
l1_dict = cand1_dict
del cand1_dict

# 순서가 없는 경우 



In [ ]:
l1_dict


In [ ]:
# l1_dict 를 구한 후
#l1_dict에서 article_id 를 가지고 순서행렬 , 순서가 없는 행렬, 을 만든다 
from itertools import combinations, permutations
permutation_list = permutations(list(cand_dict.keys()),2)


In [ ]:
# groupby 해서 t_dat 과 customer_id를 본다
# 같은 날짜 구매는 (  ), 다른 날짜라면 AB 이런식으로 연달아 ,A와 B 는 그 날과 그 다음날이다
#<> 다른 날짜여도 상관없는 패턴
#( ) 동시구매 패턴 
# https://analytics4everything.tistory.com/105 번역 
#http://simpledatamining.blogspot.com/2015/03/generalized-sequential-pattern-gsp.html 원문
# 그러니 () 로 만든 패턴은 존재 유무를 탐색하고
# <ab> 패턴은 앞문자 a의 인덱스보다 b의 인덱스가 크면 패턴으로 인지하고 추가하면 된다 

for t_dat, customerd_id 
for pattern in permutation_list:
    # 소비자 그룹별 중복을 제거했으므로 키 검사를 통해 article_id support 값 구하기
    if article_id in cand1_dict.keys():
        cand1_dict[article_id] += 1
    else:
        cand1_dict[article_id] = 1

In [ ]:
df[(df.customer_id == '00000dbacae5abe5e23885899a1fa44253a17956c6d1c3d25f88aa139fdfc657') & (df.t_dat =='2018-12-27')]

In [ ]:
for customer ,t_dat in customer_t_dat:
    print(customer, t_dat)
    print(grp_df.get_group((customer,t_dat)))
    break

In [ ]:
for customer ,t_dat in customer_t_dat:
    print(grp_df.get_group((customer,t_dat)))
    break

In [ ]:
# 중복 제거하여 소비자 그룹별 아이템 count를 1개로 제한
customer_ids = df['customer_id'].unique().tolist()

In [ ]:
cand1_dict = {}

In [ ]:
cand1_list =  list(cand1_dict.keys())

In [ ]:
# dataframe 으로 저장할 수 없는 길이이므로 모든 패턴을 지정하지 않고
# customer group에서 pattern을 찾아 list에 저장한다
len(cand1_list)

In [ ]:
len(grp_df)

In [ ]:
for idxi in range(len(cand1_list)):
    for idxj in range(len(cand2_list)):
        for group in grp_df:
            
        cand1_list[idxi] + cand1_list[idxj]

In [ ]:
%%time
min_support = 10
del_key_list = []
for article_id, support in tqdm_notebook(cand1_dict.items()):
    if support < min_support:
        del_key_list.append(article_id)

In [ ]:
for del_key in del_key_list:
    del cand1_dict[del_key]

In [ ]:
cand1_dict

In [ ]:
no_order_matrix = pd.DataFrame(columns = list(cand1_dict.keys()),index= list(cand1_dict.keys() ))

In [ ]:
no_order_matrix

In [ ]:
for article_id_i in range(len(cand1_list)):
    for article_id_i in range(len(cand1_list)):
        pd.DataFrame.iloc[i,j] =   cand1_list[article_id_i] + cand_list[article_id_j]
        

In [ ]:
len(cand1_dict.keys())

In [ ]:
from itertools import combinations, permutations

In [ ]:
no_order_pattern = list(combinations(list(cand1_dict.keys()),2))

In [ ]:
no_order_pattern